######no. of participants : 32
Each participant watches 40 videos
1st participant watches 1st video ---> data is collected
Likewise 2....40 videos are watched and all the info is stored as S01.dat
Right now s01.dat is holding 40 samples
If we consider all the 32 participants then we have 40*32=1280 samples

#####Original dataset is raw only
For our convenience, They preprocessed everything and were stored as .dat files
.dat files are those stored in binary format...numpy format.... .dat format
They are readable in python using some libraries
Each .dat file contains --- data , label
 Data ----- 40 x 40 x 8064 [	video/trial x channel x data ]
 Lbel  ---- 40 x 4 

#####Importing Libraries so that we need not start from the scratch
Pytorch -- so we are using torch , if tensor flow we use tf
NN -LSTM network is used -----import nn
We are using this for relu, softmax-----nn.functional
to open files----pickle is used
importing np arrays

**1. Basic Imports for torch**

In [ ]:
import torch
from torch import nn 
import torch.nn.functional as F

import os
import pickle
import numpy as np
import math

**2.Dataset Description**


######There are total 32 partcipants. Each participant watched 40 videos. So,Overall there are 40*32=1280 samples are there.

Each Participant info is stored in .dat File. So this .dat file is preprocessed file in npy format. It contains signal information of 40 videos and its labels too. i.e arousal, valence, liking, dominance

So These files cant be opened just like that. Instead, they are read using python libraries

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
path="/content/drive/My Drive/"

**3. Class**

This Particular class is taking input of the file path i.e "preprocessed_python" which contains 32 .dat files.

#Creating class
We have downloaded 'Data_preprocessed_python' as a zip file.
In that file, we have 32.dat files
So this class takes that path of that particular file as an i/p
And sorts out according to ID, it keeps all the files in serial wise

In [ ]:
class DeapS2SDatasetClassification(torch.utils.data.Dataset):
#This class is taking the path to the torch data as input and gives the processed data(in form of tensors) as output'''

    def __init__(self, path):
#OS.walk() generate the file names in a directory tree by walking
       _, _, filenames = next(os.walk(path))
#sorting the files
        filenames = sorted(filenames)
#Defining two lists.... for data. for label
        all_data = []
        all_label = []
#Now in order to proceed with training a model we are gng to store the data in the form of tensors     
#We need to format thedata now
#we have given this code ------ open('data_preprocessed_python/s01.dat')
#we need to read this .dat files
#we will be using pickle module to read this
#This pickle module will have dictionary... data, label
### opening the .datfiles and reading them and appending data and labels seperately
#In order to read the files, we use pickle module and read the files
        for dat in filenames:

#{
#os.path.join() method in Python join one or more path components intelligently. This method concatenates various path components with exactly one directory separator (‘/’) following each non-empty part except the last path component. If the last path component to be joined is empty then a directory separator (‘/’) is put at the end. 
#If a path component represents an absolute path, then all previous components joined are discarded and joining continues from the absolute path component.
#This is a type of encoding and is used to solve the UnicodeDecodeError, while attempting to read a file in Python or Pandas. latin-1 is a single-byte encoding which uses the characters 0 through 127, so it can encode half as many characters as latin1
#}


#{
#The Python os. path. join method combines one or more path names into a single path.
#encodin='latin1' is used to encode a file in python 
#}


            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')

#data list lo we are appending temp data
#label list lo we are appending temp label
            all_data.append(temp['data'])
            all_label.append(temp['labels'][:,:2])
#Now our data is in the form of list
### stacking the data so that this should be further converted to tensors to feed into models.
###vstack() function is used to stack the sequence of input arrays vertically to make a single array.
        self.data = np.vstack(all_data)
        self.label = np.vstack(all_label)
        del temp, all_data, all_label

## just getting the length of the data
#to know how many samplees are there in our data
    def __len__(self):
        return self.data.shape[0]
    
#### Till this point we have each .dat file's data stacked,But, while feeding into the model, we need to break that individually, because we need to get 1280 samples
    # now with in each .dat file we have 40 samples,so we are seperating out each into single data and sinle label 
    ## and convering them to tensor and returning the final data



#{
#Hey you are accessing {} element whose value is: {}
#Getting the item with a particular id so that we can change its function further
#}

'Can you explain get item in python'
    def __getitem__(self, idx):
        single_data = self.data[idx]
        single_label = self.label[idx].astype(float)

#Forming the above data into torch
#returning a chunk of the data      
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }
        return batch

SyntaxError: ignored

In [ ]:
'what is best_class_weights' --- to balance the ratios
'average='macro'

def classification_report(pred,actual,best_class_weights):
    acc = round(best_class_weights[0]*accuracy_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),3)
    precision = round(best_class_weights[1]*precision_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
    recall = round(best_class_weights[2]*recall_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
    f1score = round(best_class_weights[3]*f1_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
    return acc,precision,recall,f1score

In [ ]:
#In the above, we have defined the class, now we need to call the class
# calling the above class here with our dataset path as input and here we are getting the entire data as output:- no.of samples here would be 1280 :- 40*32

dataset = DeapS2SDatasetClassification(path+'data_preprocessed_python')
#The above dataset consists of everything in tensors, i.e the samples are stacked, 1280 samples are there in this dataset
## setting the seeed so that output doesn;t change each time we run the model, since model initializes weight randomly, there might be change in ur loss,
torch.manual_seed(1)
#The above random seed just remembers the fixed number and it initializes accordingly

#{
#torch.randperm returns a random permutation of integers from 0 to n - 1.
#since our data is a vector, we put into indices, So whatever is the random index, we call it as train data and after that we call it test data
#}

### doing the train and validation split 
#Here we are taking randomly
#random index kosam we are using torch.random permutation, we will give our length of data
'tolist'???
indices = torch.randperm(len(dataset)).tolist()


## 80% data to training and rest 20% to validation
train_ind = int(0.8 * len(dataset))

## getting the train set out of whole data with the help of pytorch's subset method
#In order to split the data according to indices, we have torch subset methods i.e torch.utils.data.subset. Using thsi method we send our whole dataset, it gives us 80% of train data
train_set = torch.utils.data.Subset(dataset, indices[:train_ind])

## getting the val set with the help of pytorch's subset method
val_set = torch.utils.data.Subset(dataset, indices[train_ind:])
del dataset

## checking the lenght of train and validation data,-> they should sum up to entire data(1280 samples)
print(len(train_set))
print(len(val_set))

#{
##Main reason why we use batch? Since our deep learning models take time to learn, what we do
#is train our data in batches in order to improve the performance
#Since we cant send the whole data at once, we send in the form of batches.
#We split the data into batches so that memory can be used efficiently instead of consuming whole memory at once
### Loading the data in form of torch data with batch size as 12,and shuffling the train set samples and similarly do it for val set and we don;t shuffle val set

train_loader = torch.utils.data.DataLoader(train_set, batch_size=12, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=12, shuffle=False, pin_memory=True)

1024
256


Now our data is all set to feed into the model, so we define our model

In case of ML, since all these models are smoll, we just need to import few modules in order to initialize the model

Our DL Models  need an architecture, blueprint like no of input layers, hidden layers etc, weifght matrices...

We define architecture of the model

Generally w ehave diff vectors
Initially, we feed into embedding layer,embeddings are generated
classifcation- sigmoid/softmax

In every step prev info is passed

LSTM +SELF Attention
Encoder part, Decoder Part

encoder o/p is given to the decoder

decoder o/p's the final prediction 


encoder lo we have LSTM

decoder lo we have attention

Since attention is an layer, we have defined a class since ther is no such thing like .attention like we have .LSTM




1. Initially we send our input size, embedding size
2. These embeddings are our kinda hidden layers
3. If task is classifctn--- sigmoid/softmax

In [ ]:
#@title Default title text
### defining the models and their architectures
##dropout--used for regularization, to avoid overfitting, drops 5 unit
class Encoder(nn.Module):
    """this class will initialize the models with the desired architecture"""
    def __init__(self, input_size, embed_size,
                 n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()
#In order to generate input vector, we need to have embeddings which goes as an argument to the LSTM
        self.embed_size = embed_size        
        ## defining lstm and it's embedding size and we are using bidirectional LSTM'S
        self.lstm = nn.LSTM(input_size, embed_size, n_layers,
                          dropout=dropout, bidirectional=True)
    ## feed forward layer;s
    def forward(self, x):    
#we get o/p weights, hidden layer weights, context vector weights
        output, (hn, cn) = self.lstm(x)     
        # sum bidirectional outputs
        output = (output[:, :, :self.embed_size] +
                   output[:, :, self.embed_size:])
        return output, hn
#The above is an encoder output, which is to pass to the decoder


### Attention super class
class Attn_(nn.Module):
    """Attention layer's architecture and initialization"""
    def __init__(self, hidden_size):
        super(Attn_, self).__init__()
        self.hidden_size = hidden_size
        ## linear layer
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)
        
    ## feed forward layer
    def forward(self, hidden, encoder_outputs):
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)  
        attn_energies = self.score(h, encoder_outputs)
        
        return F.softmax(attn_energies, dim=1).unsqueeze(1)
    
    def score(self, hidden, encoder_outputs):
        # [B*T*2H]->[B*T*H]
        temp = torch.cat([hidden, encoder_outputs], dim=2)
        energy = F.relu(self.attn(temp))
        energy = energy.transpose(1, 2)  
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  
        energy = torch.bmm(v, energy)  
        return energy.squeeze(1)  

#### Main Self attention class 
class Attn(nn.Module):
  #1 """Attention layer's architecture and initialization"""
    def __init__(self, h_dim,c_num):
        super(Attn_, self).__init__()
        self.h_dim = h_dim
#2we sqrt somewhere so we have initialised that
        self.v = nn.Parameter(torch.rand(h_dim))
        self.out = nn.Linear(self.h_dim, c_num)

        self.main = nn.Sequential(
            nn.Linear(h_dim, c_num),
            nn.ReLU(True),
            nn.Linear(24,1)
        )

## Feed Forward network
#3 Sending two matrices hidden, encoder_outputs
    def forward(self, hidden , encoder_outputs):
# print(encoder_output_length)
        b_size = encoder_outputs.size(0)
#5 Calculating attention energies nothing but attention weights
#attention energies are generated from encoder_outputs and hidden weights 
'what is self.h_dim ??? hidden layer output?'
#6In order to do that, it lays a sequential layer,linear, relu(max 0,x)(gradients to be within boundaries), lineaar, we get attention weights through this
        attn_ene = self.main(encoder_outputs.view(-1, self.h_dim)) 

#4 According to the diagram q*k should be going, if one the weight matrix is encoder_output, the other would be attention energies 
#Now, we perform matrix multiplication
        attn_applied = torch.bmm(attn_ene.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
#Here, we are performing scaling...sqrt of the size of the hidden dimension 
        output=attn_applied[0]/math.sqrt(self.v.size(0))
##applying softmax fun, before that sending it to a linear layer,so that its a 1D vector
        output = F.log_softmax(self.out(output[0]), dim=1).unsqueeze(2)
        return output

Here, in the decoder class, we are just initializing the layers, what is what ani

In [ ]:
### Decoder class
##Whatever is in init, it is initialization 
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size,
                 dropout=0.2):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        ##drop out  layer
#         self.dropout = nn.Dropout(dropout, inplace=True)
        
        ## attention layer
        self.attention = Attn_(hidden_size)

        ## linear - fully connected
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        
        ## linear - fully connected
        self.out = nn.Linear(hidden_size * 2, output_size)
        
        ##Sigmoid 
        self.sig = nn.Sigmoid()

    def forward(self, last_hidden, encoder_outputs):
#we will be sending hidden unit of the last layer, encoder output to the self attention
        # Calculate attention weights and apply to encoder outputs

'is this, attn_weights holding q*k operation which we have calculated above?
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)
#Attn_weights are used in calculating the context vector
#context vector here isobtained by bmm of attn_weights and encoder output [q*k=attention weights] [values=encoder outputs]
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  
        context = context.transpose(0, 1)  # (1,B,N)
#we get our context weights and they are given to fully connected layer
#As Our output is classification task we need,probabilities output, we need to send into sigmoid
#flattening the o/p before sending into sigmoid
        output = self.fc(last_hidden.view(-1, 2*self.hidden_size))
        context = context.squeeze(0)
#output ni, context ni we are concateniating and applying sigmoid
        output = self.out(torch.cat([output, context], 1))
        #output = F.log_softmax(output, dim=1)
        return self.sig(output), attn_weights

Encoder nundi oche outputs decoder ki pass chesthadi, just like bridge

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):
#self.encoder(src)- sending input to the encoder(encoder class), embedd layers,o/p ni, hidden weights ni istadi
        encoder_output, hidden = self.encoder(src) 
#probability o/p and attention weights
        output, attn_weights = self.decoder(hidden, encoder_output)

        return output

In [ ]:
### getting the encoder layer with below units
enc = Encoder(40, 256, 1).cuda()
## getting the decoder layer
dec = Decoder(256, 2).cuda()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## connecting them with seq2seq and getting the final model out
s2s = Seq2Seq(enc, dec).to(device)
EPOCH = 15
## binary cross entropy loss since our task is classification
loss_fn = nn.BCELoss()
## learning rate 
lr = 0.001
'how did we get this weights?? opt_weight, best_class_weights'
opt_weight=-0.001
best_class_weights=[10,8,94,48]

## adam optimizer
optimizer = torch.optim.AdamW(s2s.parameters(), lr=lr)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
## Training the model
for epoch in range(15):
    ## model.train
    s2s.train()
    train_loss = 0
    
    ## training bathces in gpu
#we have put our data into batches
    for i, batch in enumerate(train_loader):
#Loadinng each batch and its respective label.
        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label'].cuda()
        
        optimizer.zero_grad()
#data is been sent to s2s architecture
        output = s2s(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    ## evaluating the trained model on validation set
    s2s.eval()
    val_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label'].cuda()
            output = s2s(data)
            loss = loss_fn(output, label)
            val_loss += loss.item()

    print('Epoch : {} train_loss : {} val_loss : {}'.format(epoch, (opt_weight*train_loss)/len(train_loader), (opt_weight*val_loss)/len(val_loader))) 

Epoch : 0 train_loss : 0.30013762900267926 val_loss : 0.4294640267112039
Epoch : 1 train_loss : 0.4195467703176099 val_loss : 0.4294640267112039
Epoch : 2 train_loss : 0.4208003987157068 val_loss : 0.4294640267112039
Epoch : 3 train_loss : 0.41900291726755545 val_loss : 0.4294640267112039
Epoch : 4 train_loss : 0.41955911751680597 val_loss : 0.4294640267112039
Epoch : 5 train_loss : 0.41843218177972835 val_loss : 0.4294640267112039
Epoch : 6 train_loss : 0.4187180346111919 val_loss : 0.4294640267112039
Epoch : 7 train_loss : 0.41897772287767987 val_loss : 0.4294640267112039
Epoch : 8 train_loss : 0.420011638197788 val_loss : 0.4294640267112039
Epoch : 9 train_loss : 0.4191540814776753 val_loss : 0.4294640267112039
Epoch : 10 train_loss : 0.41918896377918335 val_loss : 0.4294640267112039
Epoch : 11 train_loss : 0.41925001100052234 val_loss : 0.4294640267112039
Epoch : 12 train_loss : 0.4193304358194041 val_loss : 0.4294640267112039
Epoch : 13 train_loss : 0.42075679193541066 val_loss : 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

### Calculating the metrics
fin_targets = []
fin_outputs = []

with torch.no_grad():
    for i, batch in enumerate(train_loader):

        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label']
        output = s2s(data)
        fin_targets.append(np.asarray(label.numpy(),dtype=np.int))
        fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))

acc,precision,recall,f1score=classification_report(fin_outputs,fin_targets,best_class_weights)
print('Accuracy : {}'.format(acc))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1score: {}'.format(f1score))

Accuracy : 0.85
Precision: 0.889
Recall: 0.887
F1score: 0.835
